In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit,KFold,ShuffleSplit,train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from collections import Counter
from nltk import ngrams
import numpy as np
import pandas as pd
import multiprocessing as mp
from joblib import Parallel, delayed
from tqdm import tqdm
from datetime import date, timedelta
from IPython.core.display import display, HTML
import requests, json, os, time, datetime, random, glob, re, csv, urllib, nltk, collections, pickle, random
from bs4 import BeautifulSoup as bs
from bs4.element import Tag
from dateutil.parser import parse
from stop_words import get_stop_words
from string import digits
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.util import everygrams
from nltk.tokenize import sent_tokenize, RegexpTokenizer

def rmnl(t):
    return re.sub('\s+',' ',t).strip()

def map_filenolocs(x,filelocdict):
    cleanjl = []
    for k,v in filelocdict.items():
        prfxs = [i for i in v if i in x]
        if len(prfxs) >0:
            cleanjl.append(v[0])
        else:
            continue
    return '|'.join(list(set(cleanjl)))

def get_wordcount(x):
    return len(x.split())

mainpath = str(os.getcwd())+'/'

df = pd.read_csv(mainpath +'output/4_traindataplus.csv',sep='\t')
print(df.shape)

dftraintest = df[df['outcome']!= 'nottrainingsample']
dftraintest = dftraintest.copy()
df0 = df[df['outcome']== 'nottrainingsample']

(36461, 29)


In [2]:
# dftrain = dftraintest.sample(1100)
# dftrain.to_csv(mainpath + 'output/5_dftrain.csv',sep='\t',index=False,quoting=csv.QUOTE_ALL)
dftrain = pd.read_csv(mainpath +'output/5_dftrain.csv',sep='\t')

In [3]:
dftest = dftraintest[~dftraintest['cid'].isin(dftrain['cid'].tolist())]
print(dftraintest.shape)
print(dftrain.shape)
print(dftest.shape)
print(df0.shape)

dftraintest['traintest'] = np.where(dftraintest['cid'].isin(dftrain['cid'].tolist()),'train','test')
print(dftraintest['traintest'].value_counts())

(1636, 29)
(1100, 29)
(536, 29)
(34825, 29)
train    1100
test      536
Name: traintest, dtype: int64


<h2> APPLICANTS </h2>

In [4]:
ssp = StratifiedShuffleSplit(n_splits = 5, test_size=0.3, train_size=0.7,random_state=0)
svmmodel = svm.SVC(probability=True)
cvectorizera = CountVectorizer(ngram_range=(2,3))

paramgrid = {'C': [100,10,1,0.5,0.1],
                  'gamma': [10,1, 0.1, 0.01,0.001], 
                  'kernel': ['rbf','poly','sigmoid','linear'],
                 "class_weight":["balanced",None]}

xa = dftrain['appcleantext'].tolist()
ya = dftrain['applicant'].tolist()

cvectorizera.fit(xa)
xa = cvectorizera.transform(xa)

selectora = SelectKBest(chi2, k=1000)
selectora.fit(xa, ya)
xa = selectora.transform(xa)

grida = GridSearchCV(svmmodel, paramgrid, cv=ssp, refit = True, verbose = 0, n_jobs=8) # cv=10, refit = True,
grida.fit(xa, ya)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.3,
            train_size=0.7),
             estimator=SVC(probability=True), n_jobs=8,
             param_grid={'C': [100, 10, 1, 0.5, 0.1],
                         'class_weight': ['balanced', None],
                         'gamma': [10, 1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']})

In [5]:
print(grida.best_params_)  
print(grida.best_estimator_) 

{'C': 100, 'class_weight': 'balanced', 'gamma': 10, 'kernel': 'linear'}
SVC(C=100, class_weight='balanced', gamma=10, kernel='linear', probability=True)


In [6]:
xat = dftest['appcleantext'].tolist()
yat = dftest['applicant'].tolist()

xat = cvectorizera.transform(xat)
xat = selectora.transform(xat)

predictionsa = grida.best_estimator_.predict(xat)

print(classification_report(yat, predictionsa)) 

              precision    recall  f1-score   support

           l       0.99      0.99      0.99       379
           t       0.98      0.99      0.98       157

    accuracy                           0.99       536
   macro avg       0.99      0.99      0.99       536
weighted avg       0.99      0.99      0.99       536



In [7]:
cvectorizera = CountVectorizer(ngram_range=(2,3))
selectora = SelectKBest(chi2,k=1000)

xao = dftraintest['appcleantext'].tolist()
yao = dftraintest['applicant'].tolist()

cvectorizera.fit(xao)
xao = cvectorizera.transform(xao)
selectora.fit(xao,yao)
xao = selectora.transform(xao)

svm_model_applicant = svm.SVC(C= 100, class_weight= 'balanced', gamma= 10, kernel = 'linear', probability=True)

svm_model_applicant.fit(xao, yao)

SVC(C=100, class_weight='balanced', gamma=10, kernel='linear', probability=True)

In [8]:
""" predict """

Xa = df['appcleantext'].tolist()
Xa = cvectorizera.transform(Xa)
applicant_vect = selectora.transform(Xa)

y_applicant = svm_model_applicant.predict(applicant_vect)
df['applicant'] = y_applicant 

print(df['applicant'].value_counts())

l    30427
t     6034
Name: applicant, dtype: int64


In [2]:
# tsamp = random.sample(df[df['applicant']=='t']['text'].tolist(),1)[0][:1000]
# print(tsamp)

In [31]:
with open(mainpath +'models/applicant.pkl', 'wb') as f:
    pickle.dump((cvectorizera, selectora, grida, svm_model_applicant), f)

# with open(mainpath +'models/applicant.pkl', "rb") as f:
#      cvectorizera, selectora, grida, svm_model_applicant = pickle.load(f)

<h2> OUTCOMES </h2>

In [33]:
ssp = StratifiedShuffleSplit(n_splits = 5, test_size=0.3, train_size=0.7,random_state=0)
svmmodel = svm.SVC(probability=True)
cvectorizerw = CountVectorizer(ngram_range=(2,3))

paramgrid = {'C': [100,10,1,0.5,0.1],
                  'gamma': [10,1, 0.1, 0.01,0.001], 
                  'kernel': ['rbf','poly','sigmoid','linear'],
                 "class_weight":["balanced",None]}

xw = dftrain['wincleantext'].tolist()
yw = dftrain['winner'].tolist()

cvectorizerw.fit(xw)
xw = cvectorizerw.transform(xw)

selectorw = SelectKBest(chi2, k=1000)
selectorw.fit(xw, yw)
xw = selectorw.transform(xw)

gridw = GridSearchCV(svmmodel, paramgrid, cv=ssp, refit = True, verbose = 0, n_jobs=8) # cv=10, refit = True,
gridw.fit(xw, yw)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.3,
            train_size=0.7),
             estimator=SVC(probability=True), n_jobs=8,
             param_grid={'C': [100, 10, 1, 0.5, 0.1],
                         'class_weight': ['balanced', None],
                         'gamma': [10, 1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']})

In [34]:
print(gridw.best_params_)  
print(gridw.best_estimator_) 

{'C': 10, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'rbf'}
SVC(C=10, class_weight='balanced', gamma=0.001, probability=True)


In [36]:
xwt = dftest['wincleantext'].tolist()
ywt = dftest['winner'].tolist()

xwt = cvectorizerw.transform(xwt)
xwt = selectorw.transform(xwt)

predictionsw = gridw.best_estimator_.predict(xwt)

print(classification_report(ywt, predictionsw)) 

              precision    recall  f1-score   support

           l       0.90      0.88      0.89       303
           t       0.85      0.87      0.86       233

    accuracy                           0.88       536
   macro avg       0.88      0.88      0.88       536
weighted avg       0.88      0.88      0.88       536



In [38]:
cvectorizerw = CountVectorizer(ngram_range=(2,3))
selectorw = SelectKBest(chi2,k=1000)

xaw = dftraintest['wincleantext'].tolist()
yaw = dftraintest['winner'].tolist()

cvectorizerw.fit(xaw)
xaw = cvectorizerw.transform(xaw)
selectorw.fit(xaw,yaw)
xaw = selectorw.transform(xaw)

svm_model_outcome = svm.SVC(C= 10, class_weight= 'balanced', gamma= 0.001, kernel = 'rbf', probability=True)

svm_model_outcome.fit(xaw, yaw)

SVC(C=10, class_weight='balanced', gamma=0.001, probability=True)

In [39]:
""" predict """

Xw = df['wincleantext'].tolist()
Xw = cvectorizerw.transform(Xw)
outcome_vect = selectorw.transform(Xw)

y_outcome = svm_model_outcome.predict(outcome_vect)
df['winner'] = y_outcome 

print(df['winner'].value_counts())

l    26136
t    10325
Name: winner, dtype: int64


In [1]:
# tsamp = random.sample(df[df['winner']=='t']['text'].tolist(),1)[0][-1000:]
# print(tsamp)

In [52]:
with open(mainpath +'models/outcome.pkl', 'wb') as f:
    pickle.dump((cvectorizerw, selectorw, gridw, svm_model_outcome), f)
    
# with open(mainpath +'models/applicant.pkl', "rb") as f:
#      cvectorizerw, selectorw, gridw, svm_model_outcome = pickle.load(f)

In [53]:
print(df.columns)

Index(['cid', 'date', 'member', 'memberloc', 'filenos', 'loc', 'sections',
       'text', 'otext', 'error', 'caseurl', 'file', 'errtype', 'source',
       'jltext', 'cleantext', 'judgeloc', 'membertype', 'fileno',
       'fileno_clean', 'outcome', 'idxtext', 'appcleantext', 'wincleantext',
       'applicant', 'winner', 'filenoloc', 'newloc', 'topiccleantext'],
      dtype='object')


In [55]:
selectorw_scores = pd.DataFrame(list(zip(cvectorizerw.get_feature_names_out(), selectorw.scores_, selectorw.pvalues_)), columns=['ftr', 'score', 'pval'])
selectorw_scores.sort_values(by=['score'],ascending=False, inplace= True)
selectora_scores = pd.DataFrame(list(zip(cvectorizera.get_feature_names_out(), selectora.scores_, selectora.pvalues_)), columns=['ftr', 'score', 'pval'])
selectora_scores.sort_values(by=['score'],ascending=False, inplace= True)

In [56]:
selectorw_scores.head(10)

,ftr,score,pval
211552,pay tenant,1187.468786,3.226215e-260
160321,landlord shall,1009.964294,1.225740e-221
94229,enforc offic sheriff,900.704445,6.897359e-198
198076,offic sheriff,900.704445,6.897359e-198
69602,court enforc offic,892.562617,4.060933e-196
69601,court enforc,890.197046,1.327022e-195
94225,enforc offic,883.129105,4.564456e-194
202042,order court enforc,882.263157,7.041088e-194
202041,order court,880.575880,1.638483e-193
160369,landlord shall pay,586.470520,1.467656e-129


In [57]:
selectora_scores.head(10)

,ftr,score,pval
47736,order determin,1227.756556,5.663466e-269
5106,appli order determin,1203.397657,1.114023e-263
69786,tenant appli,992.317613,8.397428e-218
69791,tenant appli order,969.494048,7.678266e-213
47826,order determin landlord,790.348777,6.767472e-174
18871,determin landlord,679.481174,8.664674e-150
55305,reason enjoy rental,559.765185,9.455819e-124
22140,enjoy rental,558.569638,1.720963e-123
22142,enjoy rental unit,556.360392,5.204281e-123
71390,tenant member,489.954426,1.457902e-108


In [58]:
selectora_scores.to_csv(mainpath+"models/5_applicantmodelfeatures.csv",sep='\t',index=False,quoting=csv.QUOTE_ALL)
selectorw_scores.to_csv(mainpath+"models/5_outcomemodelfeatures.csv",sep='\t',index=False,quoting=csv.QUOTE_ALL)

In [59]:
df.to_csv(mainpath+"output/5_model_results.csv",sep='\t',index=False,quoting=csv.QUOTE_ALL)